# RAG Sanskar Notebook

This notebook demonstrates a basic Retrieval-Augmented Generation (RAG) system setup using the `google-genai` library (Gemini LLM) together with text chunking and simple keyword-based retrieval. The notebook uses code from both the `rag_system.py` file and `requirements.txt` for dependencies.

In [ ]:
# Uncomment the next line to install dependencies directly from the notebook
%pip install -r requirements.txt

## 2. Import Libraries and Define Functions

We define functions for splitting text into chunks with overlap and for retrieving the most relevant chunks based on a simple keyword matching scheme. This setup is similar to what is in your `rag_system.py` file.

In [16]:
import re
import os

# Import the Gemini client from google-genai
from google import genai

def chunk_text(text, chunk_size=600, overlap=100):
    """
    Splits the given text into overlapping chunks using a sliding window approach.
    - text: Input string to be splitted.
    - chunk_size: The length of each chunk.
    - overlap: The number of characters overlapping between consecutive chunks.
    """
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap)]

def find_relevant_chunks(query, chunks, top_k=3):
    """
    Finds and returns the indices of the chunks that have the highest keyword overlap with the query.
    This is a simple method to score relevance based on matching words.
    """
    # Tokenize query into words (converted to lower case for uniformity)
    query_words = set(re.findall(r'\w+', query.lower()))
    scores = []
    
    for i, chunk in enumerate(chunks):
        # Tokenize chunk into words
        chunk_words = set(re.findall(r'\w+', chunk.lower()))
        # Score is the count of common words between query and chunk
        score = len(query_words & chunk_words)
        scores.append((score, i))
    
    # Sort chunks by score in descending order and get indices of the top scoring chunks
    return [i for _, i in sorted(scores, reverse=True)[:top_k]]

## 3. Prepare Document and Process Text

Here we define a sample document (a whimsical story) and perform basic text cleaning by removing extra spaces. We then create chunks from this cleaned text.

In [17]:
# Define the sample document
document = """
Once upon a time, in a whimsical land called Veggieville, there lived a curious rabbit named Bunny. Bunny was no ordinary rabbit—she had a knack for finding strange and magical objects. One sunny morning, while hopping through the Enchanted Forest, she stumbled upon a peculiar hat lying under a giant carrot-shaped tree. The hat was no ordinary hat—it was the Magic Hat of 2000 Lines, a legendary artifact said to grant its wearer the power to weave stories, spells, and songs with just a thought.
Bunny picked up the hat and examined it closely. It was a tall, floppy hat with shimmering silver threads that seemed to dance in the sunlight. As she placed it on her head, a voice boomed from nowhere and everywhere at once.
"Ah, a new wearer! Welcome, Bunny. I am the Hat, and I am bound to serve you. But beware—my magic is not infinite. I can only create 2000 lines of magic before my power fades. Use them wisely!"
Bunny's ears perked up. "2000 lines? That's a lot! What can I do with them?"
The Hat chuckled. "Anything you can imagine! Poems, riddles, spells, even entire stories. But remember, once the lines are used up, my magic is gone forever."
Excited, Bunny decided to test the Hat's powers. She thought of her best friend, Carrot, a cheerful orange vegetable with a knack for getting into trouble. "Hat, can you tell me a story about Carrot?"
The Hat glowed, and a story began to unfold:
Once, in the heart of Veggieville, there lived a carrot named Carrot who loved to explore. One day, he stumbled upon a salty cave guarded by a grumpy old grain named Salt. Salt was the keeper of the Cave of Crystals, a place filled with shimmering treasures. But Salt was lonely and bitter, and he refused to let anyone enter.
Carrot, being the curious soul he was, decided to befriend Salt. He brought him a basket of fresh vegetables and sang him a song so sweet that even Salt's hard exterior began to melt. Slowly, Salt opened up and shared his treasures with Carrot, and the two became the unlikeliest of friends.
Bunny clapped her paws. "That was amazing! But... how many lines did that use?"
The Hat sighed. "That was 15 lines. You have 1985 left."
Bunny gasped. "Oh no! I need to be more careful. I don’t want to waste your magic."
"""

# Clean the text by replacing multiple whitespace with a single space
clean_text = re.sub('\s+', ' ', document).strip()

# Split the cleaned text into chunks using the sliding window approach
chunks = chunk_text(clean_text)

# Print the first few chunks for verification
print("First 2 chunks:")
print(chunks[:2])

First 2 chunks:
['Once upon a time, in a whimsical land called Veggieville, there lived a curious rabbit named Bunny. Bunny was no ordinary rabbit—she had a knack for finding strange and magical objects. One sunny morning, while hopping through the Enchanted Forest, she stumbled upon a peculiar hat lying under a giant carrot-shaped tree. The hat was no ordinary hat—it was the Magic Hat of 2000 Lines, a legendary artifact said to grant its wearer the power to weave stories, spells, and songs with just a thought. Bunny picked up the hat and examined it closely. It was a tall, floppy hat with shimmering silver thr', 'unny picked up the hat and examined it closely. It was a tall, floppy hat with shimmering silver threads that seemed to dance in the sunlight. As she placed it on her head, a voice boomed from nowhere and everywhere at once. "Ah, a new wearer! Welcome, Bunny. I am the Hat, and I am bound to serve you. But beware—my magic is not infinite. I can only create 2000 lines of magic 

## 4. Set Up the Gemini Client and Execute a Query

In this step, we set up the Gemini LLM client using an API key stored as an environment variable. Then we perform a simple retrieval-based generation:

- We ask the user for a query.
- We retrieve the top relevant chunks using our keyword matching function.
- We then pass the retrieved context and query to Gemini and print the LLM's answer.

> **Note:** In a notebook, interactive input might not work as expected. For demonstration purposes, you can set a sample query string.

In [18]:
# Set up the Gemini client using the API key from an environment variable
# Ensure that you have set GEMINI_API_KEY in your environment for this cell to work
from dotenv import load_dotenv

load_dotenv()
client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

# For demonstration, we define a sample query.
sample_query = "What magic does the Hat have?"

# 1. Retrieve the indices of the most relevant chunks based on the sample query
indices = find_relevant_chunks(sample_query, chunks)

# 2. Print the relevant chunks
print("\nRelevant chunks:")
for i in indices:
    print(f"Chunk {i}: {chunks[i]}")

# 3. Create a single context string from the retrieved chunks
context = " ".join([chunks[i] for i in indices])

# 4. Build the prompt for Gemini (including context and question)
prompt = f"Context: {context}\nQuestion: {sample_query}\nAnswer:"

# 5. Use Gemini to generate an answer based on the prompt
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

# 6. Print the answer
print("\nAnswer:", response.text)


Relevant chunks:
Chunk 4: keliest of friends. Bunny clapped her paws. "That was amazing! But... how many lines did that use?" The Hat sighed. "That was 15 lines. You have 1985 left." Bunny gasped. "Oh no! I need to be more careful. I don’t want to waste your magic."
Chunk 1: unny picked up the hat and examined it closely. It was a tall, floppy hat with shimmering silver threads that seemed to dance in the sunlight. As she placed it on her head, a voice boomed from nowhere and everywhere at once. "Ah, a new wearer! Welcome, Bunny. I am the Hat, and I am bound to serve you. But beware—my magic is not infinite. I can only create 2000 lines of magic before my power fades. Use them wisely!" Bunny's ears perked up. "2000 lines? That's a lot! What can I do with them?" The Hat chuckled. "Anything you can imagine! Poems, riddles, spells, even entire stories. But remember, 
Chunk 2: Hat chuckled. "Anything you can imagine! Poems, riddles, spells, even entire stories. But remember, once the line

## Summary

In this notebook we:

- **Imported required libraries** and set up helper functions for text chunking and retrieval.
- **Cleaned and split a sample document** into overlapping chunks.
- **Used a custom Gemini LLM client setup** to generate an answer from a sample query based on the retrieved context.

This simple example illustrates how to combine retrieval techniques with LLM-powered generation in a notebook environment while providing full documentation and inline comments.